In [1]:
from datetime import datetime
import urllib.request
import pandas as pd
import json as jsn
import time
import sys

In [2]:
# main variables
with open('vk_access_token.txt', encoding='utf-8') as txt:
    token = txt.readline() # replace with .env mechanic later
vk_tail = f'&v=5.95&access_token={token}'
vk_page = 150476436 # which page id to scan for users and their data
retry_limit = 10
debug = ''

In [3]:
# Secondary functions
# place a dttm mark of choice
def time_mark(my_type='time', prefix='', mode='print'):
    options = {'date':[0,10], 'datetime':[0,19], 'full':[0,26], 'time':[11,19], 'fulltime':[11,26]}
    try:
        a = options[my_type]
    except:
        a = options['full']
    output = f'{prefix}{str(datetime.now())[a[0]:a[1]]}'
    if mode=='print':
        print(output)
    else:
        return output
# to see how much of the work is done
def progress_bar(done=0, total=1, width=10, prefix='', postfix='', show_completion=True):
    done_share = (total if done > total else done)/total
    done_bars = round(width*done_share)
    completion = f' {round(done_share*100,1)} %' if show_completion else ''
    sys.stdout.write(f'\r{prefix}[{"@"*done_bars}{"-"*(width-done_bars)}]{completion}{postfix}{" "*999}')
# check if object has a key
def check_key(obj, key, mode=''):
    if key in obj:
        if mode=='value':
            return obj[key]
        else:
            return 1
    else:
        return 0
# <= end of => check_key
time_mark('datetime', 'Finish: ')

Finish: 2020-11-18 17:38:56


In [4]:
%%time
# collect all users of a page
# time_asc = by time of joining the group, Moderator status needed // id_asc instead
text = f'group_id={vk_page}&sort=id_asc&offset=0&count=1000{vk_tail}'
url_prime = f'https://api.vk.com/method/groups.getMembers?{text}'
users, offset, do_we_go = [], 0, True
while do_we_go:
    for n_try in range(0, retry_limit): # several tries if somethings fucked
        time.sleep(0.5) # sleep before each load to avoid api limits
        url = url_prime.replace('&offset=0&', f'&offset={offset}&')
        try:
            with urllib.request.urlopen(url) as url_request:
                json = jsn.loads(url_request.read().decode())
        except:
            json = ''
            print(f"Error: {url} - didn't load json", 0)
        if 'response' in json:
            users.extend(json['response']['items'])
            if len(users) < json['response']['count']:
                offset += 1000
            else:
                do_we_go = False
            break
        elif 'error' in json:
            print(f'Error: {url} - {json["error"]["error_msg"]}')
        else:
            print(f'Error: {url} - no response')
print(f'{len(users)} => {users[:10]} ...')

4001 => [161, 793, 1404, 1649, 2237, 2416, 3155, 3461, 4556, 4716] ...
Wall time: 2.99 s


In [5]:
%%time
get_user_cols = ['id', 'first_name', 'last_name'
    , 'deactivated', 'is_closed', 'bdate', 'city.title'
    , 'country.title', 'education' # only check if it exists
    , 'followers_count', 'games' # only check if it exists
    , 'has_mobile', 'has_photo', 'home_town', 'last_seen.time'
    , 'last_seen.platform', 'nickname', 'occupation.type'
    , 'personal.political', 'personal.langs', 'personal.people_main'
    , 'personal.life_main', 'personal.smoking', 'personal.alcohol'
    , 'relation', 'sex', 'site', 'trending', 'verified', 'wall_default'
]
user_cols_for_api = get_user_cols[:]
for i in range(0, len(get_user_cols)):
    user_cols_for_api[i] = get_user_cols[i].split('.')[0]
user_cols_for_api = list(dict.fromkeys(user_cols_for_api))
user_cols_for_api = ','.join(map(str, user_cols_for_api))
# get dict of data out of user json row
def parse_single_user(obj):
    global get_user_cols
    row = {}
    for s in get_user_cols:
        if s in ['connections', 'contacts', 'education', 'games']:
            row[s] = check_key(obj, s)
        elif s in ['personal.langs']:
            try:
                row[s] = len(obj['personal']['langs'])
            except:
                row[s] = 0
        elif '.' in s:
            if s.split('.')[0] in obj:
                row[s] = check_key(obj[s.split(".")[0]], s.split(".")[1], 'value')
            else:
                row[s] = 0
        else:
            row[s] = check_key(obj, s, 'value')
    return row
# <= end of => parse_get_wall_row
# Convert json to users dataframe
def parse_users_json(obj):
    global get_user_cols
    df = pd.DataFrame(columns=get_user_cols)
    for a in obj:
        single_user = parse_single_user(a)
        df = df.append(
            pd.DataFrame(single_user, index=[0])
            , ignore_index=True
            , sort=False
        )
    return df
# <= end of => parse_users_json
# return info df of users given (not more than 1000)
def get_a_pile_of_users(s_users):
    global user_cols_for_api, debug
    s_text = f'user_ids={s_users}&fields={user_cols_for_api}{vk_tail}'
    url = f'https://api.vk.com/method/users.get?{s_text}'
    user_num = len(s_users.split(','))
    try:
        with urllib.request.urlopen(url) as url_request:
            my_json = jsn.loads(url_request.read().decode())
            debug = my_json
            if 'response' in my_json:
                user_loaded_num = len(my_json['response'])
                if user_num == user_loaded_num:
                    my_df = parse_users_json(my_json['response'])
                else:
                    return f'Error: {url} - wrong response', 0
            else:
                return f'Error: {url} - no response', 0
        return my_df, 1
    except:
        return f'Error: {url} - didn\'t load json', 0
# <= end of => get_a_pile_of_users
# get df of users data
def get_all_users(user_ids):
    global retry_limit, get_user_cols
    tm_start = time_mark('time','','val')
    user_num = len(user_ids)
    if user_num<=0:
        print(f'get_all_users - no data - {time_mark("full","","val")}')
        return('Error: no data')
    else:
        progress_bar(0, 100, 20
            , prefix=f'{tm_start} => {time_mark("time","","val")} => '
            , postfix=f' => get_all_users initiated')
    df_users = pd.DataFrame(columns=get_user_cols)
    max_offset = user_num // 100 + 1 if user_num % 100 > 0 else user_num // 100
    for n_offset in range(0, max_offset):
        s_users = ','.join(map(str, user_ids[(n_offset*100):((n_offset+1)*100)]))
        for n_try in range(0, retry_limit): # several tries if somethings fucked
            time.sleep(0.5) # sleep before each load to avoid api limits
            df_tmp, debug_code = get_a_pile_of_users(s_users)
            if debug_code > 0:
                df_users = df_users.append(df_tmp, ignore_index=True, sort=False)
                break
            #else:
                #print(f'Did not load json')
        # pile of users loaded, show progress bar
        progress_bar(n_offset, max_offset, 20
            , prefix=f'{tm_start} => {time_mark("time","","val")} => '
            , postfix=f' => {n_offset} / {max_offset} user piles processed')
    # all comments are loaded
    progress_bar(1, 1, 20
        , prefix=f'{tm_start} => {time_mark("time","","val")} => '
        , postfix=f' => all data from {max_offset} user piles are loaded')
    return df_users
# <= end of => get_all_users
df_users = get_all_users(users)

17:39:32 => 17:40:21 => [@@@@@@@@@@@@@@@@@@@@] 100.0 % => all data from 41 user piles are loaded                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [6]:
df_users.head(3)

,id,first_name,last_name,deactivated,is_closed,bdate,city.title,country.title,education,followers_count,...,personal.people_main,personal.life_main,personal.smoking,personal.alcohol,relation,sex,site,trending,verified,wall_default
0,161,Valentina,Savosina,0,False,0,Saint Petersburg,Russia,0,20699,...,0,0,0,0,0,1,https://vk.com/prekrasnoe_riadom https://vk.co...,0,0,all
1,793,Roman,Sidorov,0,False,17.6,Saint Petersburg,Russia,0,692,...,0,0,0,0,0,2,,0,0,all
2,1404,Katerina,Smirnova,0,False,0,Saint Petersburg,Russia,0,1935,...,0,0,0,0,0,1,,0,0,all


In [7]:
df_users.to_csv(f'users_{vk_page}.csv', encoding='utf-8')

In [8]:
%%time
# collect all subscriptions
i = 0
final_items = []
tm_start = time_mark('time','','val')
progress_bar(0, 100, 20
    , prefix=f'{tm_start} => {time_mark("time","","val")} => '
    , postfix=f' => get_all_subs initiated')
for user in users:
    s_text = f'user_id={user}&extended=1&offset=0&count=200&fields=followers_count,members_count{vk_tail}'
    url_prime = f'https://api.vk.com/method/users.getSubscriptions?{s_text}'
    items, offset, do_we_go = [], 0, True
    while do_we_go:
        for n_try in range(0, retry_limit): # several tries if somethings fucked
            url = url_prime.replace('&offset=0&', f'&offset={offset}&')
            try:
                with urllib.request.urlopen(url) as url_request:
                    json = jsn.loads(url_request.read().decode())
            except:
                json = ''
                print(f"Error: {url} - didn't load json", 0)
            if 'response' in json:
                if len(json['response']['items']) > 0:
                    items.extend(json['response']['items'])
                else:
                    do_we_go = False
                    break
                if len(items) < json['response']['count']:
                    offset += 200
                else:
                    do_we_go = False
                    for item in items:
                        item['owner'] = user
                    final_items.extend(items)
                break
            elif 'error' in json:
                if json['error']['error_msg'] == 'This profile is private':
                    do_we_go = False
                    break
                else:
                    print(f'Error: {url} - {json["error"]["error_msg"]}')
                    do_we_go = False
                    break
            else:
                print(f'Error: {url} - no response')
                do_we_go = False
                break
            time.sleep(0.5) # sleep before each load to avoid api limits
        progress_bar(i, len(users), 20
            , prefix=f'{tm_start} => {time_mark("time","","val")} => '
            , postfix=f' => {i} / {len(users)} subs / {len(final_items)} collected / {user}-{offset}')
    i += 1
progress_bar(1, 1, 20
    , prefix=f'{tm_start} => {time_mark("time","","val")} => '
    , postfix=f' => all data from {len(users)} subs piles are loaded / {len(final_items)} itemms collected')

17:44:18 => 18:07:55 => [@@@@@@@@@@@@@@@@@@@@] 100.0 % => all data from 4001 subs piles are loaded / 498286 itemms collected                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [9]:
len(final_items)

498286

In [10]:
%%time
df = pd.DataFrame(columns=['owner', 'id', 'type', 'name', 'subs'])
data = []
tm_start = time_mark('time','','val')
i = 0
for item in final_items:
    row = {}
    row['owner'] = item['owner']
    row['id'] = item['id']
    row['type'] = item['type']
    if item['type'] == 'page':
        if 'name' in item:
            row['name'] = item['name']
        else:
            row['name'] = ''
        if 'members_count' in item:
            row['subs'] = item['members_count']
        else:
            row['subs'] = -1
    elif item['type'] == 'profile':
        if 'first_name' in item:
            row['name'] = item['first_name']
        else:
            row['name'] = ''
        if 'last_name' in item:
            if len(row['name']) > 0:
                row['name'] += ' '
            row['name'] += item['last_name']
        if 'followers_count' in item:
            row['subs'] = item['followers_count']
        else:
            row['subs'] = -1
    elif item['type'] == 'group':
        if 'name' in item:
            row['name'] = item['name']
        else:
            row['name'] = ''
        if 'members_count' in item:
            row['subs'] = item['members_count']
        else:
            row['subs'] = -1
    else:
        print(f"Unknown type: {item['type']}")
        break
    data.append(row)
    i += 1
    progress_bar(i, len(users), 20
        , prefix=f'{tm_start} => {time_mark("time","","val")} => '
        , postfix=f' => {i} / {len(final_items)} subs')
df = pd.DataFrame(data)
df.head(5)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



18:24:49 => 18:24:49 => [@@@@@@@@@@@@@@@@@@@@] 100.0 % => 6122 / 498286 subs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
df.type.value_counts()

page       461582
profile     34121
group        2583
Name: type, dtype: int64

In [12]:
df.head()

,id,name,owner,subs,type
0,59245439,Винтажная мода,161,564610,page
1,33242504,Вязание крючком,161,547983,page
2,92307020,"Тамара, какого хрена?",161,77990,page
3,29686754,Шпильки | Женский Журнал,161,3803523,page
4,182177005,"Афиша Мам: встречи с экспертами в СПб, чаты мам",161,1804,page


In [13]:
df.to_csv('subs_{vk_page}.csv', encoding='utf-8')

In [14]:
df_users.rename(columns={'id':'owner'}, inplace=True)
df_users.owner = df_users.owner.astype(int)
df_users.head(3)

,owner,first_name,last_name,deactivated,is_closed,bdate,city.title,country.title,education,followers_count,...,personal.people_main,personal.life_main,personal.smoking,personal.alcohol,relation,sex,site,trending,verified,wall_default
0,161,Valentina,Savosina,0,False,0,Saint Petersburg,Russia,0,20699,...,0,0,0,0,0,1,https://vk.com/prekrasnoe_riadom https://vk.co...,0,0,all
1,793,Roman,Sidorov,0,False,17.6,Saint Petersburg,Russia,0,692,...,0,0,0,0,0,2,,0,0,all
2,1404,Katerina,Smirnova,0,False,0,Saint Petersburg,Russia,0,1935,...,0,0,0,0,0,1,,0,0,all


In [16]:
df_final = df_users.merge(df, on='owner', how='left')
df_final.head(5)

,owner,first_name,last_name,deactivated,is_closed,bdate,city.title,country.title,education,followers_count,...,relation,sex,site,trending,verified,wall_default,id,name,subs,type
0,161,Valentina,Savosina,0,False,0,Saint Petersburg,Russia,0,20699,...,0,1,https://vk.com/prekrasnoe_riadom https://vk.co...,0,0,all,59245439.0,Винтажная мода,564610.0,page
1,161,Valentina,Savosina,0,False,0,Saint Petersburg,Russia,0,20699,...,0,1,https://vk.com/prekrasnoe_riadom https://vk.co...,0,0,all,33242504.0,Вязание крючком,547983.0,page
2,161,Valentina,Savosina,0,False,0,Saint Petersburg,Russia,0,20699,...,0,1,https://vk.com/prekrasnoe_riadom https://vk.co...,0,0,all,92307020.0,"Тамара, какого хрена?",77990.0,page
3,161,Valentina,Savosina,0,False,0,Saint Petersburg,Russia,0,20699,...,0,1,https://vk.com/prekrasnoe_riadom https://vk.co...,0,0,all,29686754.0,Шпильки | Женский Журнал,3803523.0,page
4,161,Valentina,Savosina,0,False,0,Saint Petersburg,Russia,0,20699,...,0,1,https://vk.com/prekrasnoe_riadom https://vk.co...,0,0,all,182177005.0,"Афиша Мам: встречи с экспертами в СПб, чаты мам",1804.0,page


In [18]:
%%time
df_final.to_csv(f'users_n_subs_{vk_page}.csv', encoding='utf-8')